In [54]:
import sys
sys.path.append("..")

from pymongo import MongoClient
import pandas as pd
import numpy as np
from data_transformation.db_env import DbEnv, db
from data_modeling.nlp_modeling import NLPModeling
from bertopic import BERTopic
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from dateutil.relativedelta import relativedelta
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

In [55]:
conn, cursor = DbEnv().connect_sql()

sql = "SELECT token, tag, date, doc_num FROM newssentoken WHERE artist ='%s'" % ('뮤직카우')
# sql = "SELECT token, tag, date FROM newssentoken"
cursor.execute(sql)
conn.commit()
tuple_sql = cursor.fetchall()
array_token = np.array(tuple_sql)

In [56]:
list_token_temp = list(map(lambda x: np.asarray(x[0].split(', ')), tuple_sql))
list_tag = list(map(lambda x: np.asarray(x[1].split(', ')), tuple_sql))
list_token = list(map(lambda x, y: np.stack((x, y), axis=1), list_token_temp, list_tag))

list_tokens = list(map(lambda x: x[(x[:, 1] == 'NNP') | (x[:, 1] == 'NNG')][:, 0].tolist(), list_token))
list_tokens = list(map(lambda x: ' '.join(x), list_tokens))

list_time = array_token[:, 2]

In [57]:
list_tokens

['셀 럽 이슈 뮤직카우 규제 리스크 불신 뮤직카우 대표 정현경 김지수 플랫 폼 업계 시선 뮤직카우 음원 소비 팝 시장 조각 투자 저작 관 심도 음반 제작 관련자 일반',
 '돈 증권 토큰 시장 본격 개막 미래 선두 펀 블 부동산 카사 부동산 뮤직카우 음악 저작권료 미술품 분할 소유 플랫 폼 서비스 준비 이랜드 넥스트 이노 플 테사 협력 관계 구축 올해 연초 한국 정보 인증 블록체인 전문 기업',
 '선점 전쟁 서막 키움증권 초반 주도 이후 혁신 금융 서비스 지정 펀 블 부동산 카사 부동산 뮤직카우 음악 저작권료 협력 미술품 분할 소유 플랫 폼 서비스 준비 이랜드 넥스트 이노 플 테사 체결 투자처',
 '아트 테크 바람 아트 증권사 교보 증권 아트 테크 플랫 폼 테사 투자 진행 키움증권 테사 뮤직카우 업무 협약 아트 테크 사업 진출 신한 투자 증권 서울 옥션 블루 미술품 조각 투자 플랫폼 조각 투자',
 '문화 금융 생태 뮤직카우 아티스트 응원 아티스트 팬 특별 저작 공유 문화 음악 저작 대중 관심 저작 인식 개선 노력 뮤직카우 이용자 편의 즐거움 뮤직카우',
 '백전 무패 국내 상륙 애플 페이 뮤직카우 음악 저작 카사 펀 블 소유 부동산 뮤직카우 수익 증권 조각 투자 증권 형태 롤린 상장 상승 저작권료 참여 청구',
 '인터뷰 아트 지주 희 정태완 웹 첨단 서비스 소셜 플 뮤직카우 음원 카사 부동산 조각 투자 서비스 성공 지난해 조각 투자 업체 혁신 금융 서비스 신청 미술품 한우 조각 투자 투자 계약 증권 증권',
 '임창정 소주 잔 이름 소주 출시 론 제품 애정 한편 저작권료 거래 플랫 폼 뮤직카우 공개 임창정 소주 잔 곡 저작권료',
 '리 테일 키움증권 증권 토큰 시장 왕좌 키움증권 부동산 브릭 펀 블 카사 음악 저작 뮤직카우 미술품 이랜드 넥스트 테사 자체 플랫 폼 페어 스퀘어 랩 다양 분야 업체 협력 박혜진 대신 증권 연구원 키움증권 뮤직카우',
 '회원 거래 규모 음악 저작 시장 성장 뮤직카우 뮤직카우 이용자 편의 즐거움 뮤직카우 시대 예고 아티스

In [62]:
dictionary = Dictionary([doc.split() for doc in list_tokens])

In [58]:
# topic_num = proper_topic_num()
topic_num = round(len(list_tokens) ** 0.25)

In [59]:
vectorizer = CountVectorizer(input=list_tokens, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer, nr_topics=topic_num, top_n_words=10, calculate_probabilities=True)


In [75]:
# topics, _ = model.fit_transform(list_tokens)
model.fit(list_tokens)

In [87]:
topics=[[x[0] for x in model.get_topic(topic)] for topic in range(len(model.get_topics()))[:-1]]

In [88]:
topics

[['뮤직카우', '투자', '베스트', '조각', '유니콘', '현재', '증권', '플랫', '신사', '블리'],
 ['증권', '투자', '뮤직카우', '키움증권', '토큰', '금융', '조각', '서비스', '업무', '협약']]

In [91]:
coherence_model = CoherenceModel(
    topics=[[x[0] for x in model.get_topic(topic)] for topic in range(len(model.get_topics()))[:-1]],    texts=[doc.split() for doc in list_tokens],
    dictionary=dictionary,
    coherence='c_v',
    topn=10
)
coherence_scores = coherence_model.get_coherence_per_topic()
agg_score = coherence_model.aggregate_measures(coherence_scores)

In [95]:
print(coherence_scores)
agg_score

[0.2726634290539248, 0.599485569122189]


0.4360744990880569

In [51]:
repre_docs = model.representative_docs

In [53]:
repre_docs[3]

['경북 과학 영재교육 원 최고상 과기 정통부 장관 개상 수상 북대 류 창빈 이예준 정성우 학생 팀 수학 분야 최고상 과기 정통부 장관 상 강수정 신정훈 차 유진 학생 팀 생물 분야 과학 기술원 장상 류 언 신석 이동윤 이채훈',
 '영남 공대 대구대 계명 문화 대 경북 대 대학 소식 경북 류 창빈 성광 이예준 성중 정성우 침산 학생 팀 수학 분야 최고상 과학 기술 정보 통신부 장관 상 경북 수학 양승엽 교수 지도 팀 수학 루빅스 큐브',
 '군 백기 男 아이돌 통법 가요 공감 지난해 사회 복무 요원 대체 복무 그룹 엑소 백현 대표 소통 왕 팬 근황 진이 입소 사단 신병교육대 훈련병 휴대폰 사용 시범 부대 선정',
 '이선희 경북 의원 대한민국 지방 자치 평가 의정 정책 비전 대상 최우수 지방 자치 활성 공로 의정 정책 비전 대상 부문 최우수 수상 이선희 경북 의원 청도 이선희 의원 경북 의회 초대 예산 결산 특별 위원장 기획 경제 위원회 위원 선진 지방 회구 현',
 '이선희 경북 의원 대한민국 지방 자치 평가 의정 정책 비전 대상 최우 수상 사진 경북 의회 제공 이선희 경북 의원 청도 대한민국 지방 자치 평가 의정 정책 비전 대상 이선희 의원 지방 자치 발전 도민 목소리 지방 정부 운영 반영 가능 일',
 '이선희 경북 의원 대한민국 지방 자치 평가 의정 정책 비전 대상 최우수 수상 이선희 경북 의원 청도 국민 힘 대한민국 지방 자치 평가 의정 정책 비전 대상 경북 의회 제공 이선희 경북 의원 청도 국민 힘 사진 대한민국 지방 자치 평가 의정 정책 비전 대상',
 '가수 김종국 각별 안양사 고향 사랑기 부제 동참 가수 김종국 최근 고향 사랑 음 안양시 고항 사랑기 부제 동참 안양시 안양 명학 초등 학교 신성 고등학교 졸업 안양 김종국 인조 보이 그룹 터보 음악',
 '가수 김종국 안양시 고향 사랑기 부제 동참 사진 안양시 가수 김종국 최근 고향 사랑 음 안양시 고항 사랑기 부제 동참 안양 명학 초등 학교 신성 고등학교 졸업 안양 김종국 인조 보이 그룹 터보

In [48]:
df_time = model.topics_over_time(list_tokens, topics, list_time, nr_bins=20)

In [49]:
fig = model.visualize_topics_over_time(df_time, top_n_topics=10, width=900)

In [50]:
fig